In [ ]:
from transformers import BertTokenizer, TFBertForMaskedLM
import tensorflow as tf
import os
import numpy as np
import re
import matplotlib.pyplot as plt


In [ ]:
tokenizer = BertTokenizer.from_pretrained('sharif-dal/dal-bert')
model = TFBertForMaskedLM.from_pretrained('sharif-dal/dal-bert',from_pt=True)
model.summary()


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForMaskedLM: ['bert.embeddings.position_ids', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing TFBertForMaskedLM from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForMaskedLM from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


Model: "tf_bert_for_masked_lm_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  115975680 
                                                                 
 mlm___cls (TFBertMLMHead)   multiple                  31553344  
                                                                 
Total params: 116607808 (444.82 MB)
Trainable params: 116607808 (444.82 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
def clean_text(line):
    line = re.sub(r'-+',' ',line)
    line = re.sub(r'/',' ',line)
    line = re.sub(r'[a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += "."
    return line


In [ ]:
import pandas as pd
df = pd.read_excel('exportStatements.xlsx')

datas = df['inFormalWords'].tolist()
datas = list(filter(lambda x: isinstance(x, str), datas))



In [ ]:
# datas[41655]
text_lst = []
len_lst = []
for idx , line in enumerate(datas):
    # print(idx)
    if len(line.split("/")) >=15:
        line = clean_text(line)
        text_lst.append(line)
        len_lst.append(len(line.split("/")))

In [ ]:
text_lst

['اینو دید داداشم رو میزم و گفت باز بی اجازه در کمد منو وا کردی و دس به کتابای من زدی.',
 'با یه تاکسی لکنته دختر همسایه دم رو غروب رسوندیم با مامانم بیمارستان همه صورتشو خون پوشونده بود و همش ناله می کرد.',
 'بهداشتی خودتم میدونی امکانش نیست ولی فقط با رعایت اصول میتونیم کمک کنیم که بره بیماری و شیوع کمتر بشه.',
 'کمترین میزان صدای از همه نظر ولی عالیه مکالمه در ولوم که اینه ایرادش بزرگترین انگار روی پخش کنه هم.',
 'دادم چون تو رنگی هم که سفارش دادم همینجوری سفارش عکساش فقط سفیدش بود و نمونه دیگشو نداشت رنگای.',
 'خریدم من تا اینکه برا کسایی که شغلشون زرگری ولی می\u200cکشید کیلو دو بیشتر هست بد نیست.',
 'شد که خانواده ش رضا کلی بهش برخورد ولی دیگه همون اون حرفا رو تکرار نکردن.',
 'آدم های تا خسته و نا امید میشم به خودم میگم همه خوشحال موفق هم یه جا شدن ولی پا ادامه دادن.',
 'چون منم خیلی شما اول نمیدونستم از مثل چی میخوام خواسته هام رو غیر ممکن میدیدم و همه درهای رسیدن بسته.',
 'خر کار بگیر بخواب که صبح باید بیدار شی مثل کنی و حالا حالا ها تا سر کوچه هم\u200c نمیتونی بری یه چایی تلخ ه

In [ ]:
len(text_lst)

10542

In [ ]:
inputs = tokenizer(text_lst,max_length=100,truncation=True,padding='max_length',return_tensors='tf')

print(inputs.keys())


dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [ ]:
inputs

{'input_ids': <tf.Tensor: shape=(10542, 100), dtype=int32, numpy=
array([[    2,  6981,  2316, ...,     0,     0,     0],
       [    2,  1888,  2482, ...,     0,     0,     0],
       [    2,  4570, 20435, ...,     0,     0,     0],
       ...,
       [    2, 18137,  1951, ...,     0,     0,     0],
       [    2,  2464,  2832, ...,     0,     0,     0],
       [    2,  2482,  3827, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(10542, 100), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(10542, 100), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype

In [ ]:
inputs['labels'] = inputs['input_ids']
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [ ]:
inp_ids = []
lbs = []
idx = 0
for inp in inputs.input_ids.numpy():
    actual_tokens = list(set(range(100)) -
                         set(np.where((inp == 101) | (inp == 102)
                            | (inp == 0))[0].tolist()))
    num_of_token_to_mask = int(len(actual_tokens)*0.15)
    token_to_mask = np.random.choice(np.array(actual_tokens),
                                     size=num_of_token_to_mask,
                                     replace=False).tolist()
    inp[token_to_mask] = 103
    inp_ids.append(inp)
    idx += 1
inp_ids = tf.convert_to_tensor(inp_ids)
inputs['input_ids'] = inp_ids


In [ ]:
inputs.input_ids

<tf.Tensor: shape=(10542, 100), dtype=int32, numpy=
array([[    2,  6981,  2316, ...,     0,     0,     0],
       [    2,   103,  2482, ...,     0,     0,     0],
       [    2,  4570, 20435, ...,     0,     0,     0],
       ...,
       [    2, 18137,  1951, ...,     0,     0,     0],
       [    2,  2464,  2832, ...,     0,     0,     0],
       [    2,  2482,  3827, ...,     0,     0,     0]], dtype=int32)>

In [ ]:
inputs.attention_mask

<tf.Tensor: shape=(10542, 100), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32)>

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
history = model.fit([inputs.input_ids,inputs.attention_mask],inputs.labels,verbose=1,batch_size=8,epochs=10)

Epoch 1/10
1318/1318 [==============================] - 364s 243ms/step - loss: 0.2098
Epoch 2/10
1318/1318 [==============================] - 323s 245ms/step - loss: 0.0723
Epoch 3/10
1318/1318 [==============================] - 323s 245ms/step - loss: 0.0288
Epoch 4/10
1318/1318 [==============================] - 323s 245ms/step - loss: 0.0160
Epoch 5/10
1318/1318 [==============================] - 323s 245ms/step - loss: 0.0149
Epoch 6/10
1318/1318 [==============================] - 323s 245ms/step - loss: 0.0141
Epoch 7/10
1318/1318 [==============================] - 323s 245ms/step - loss: 0.6036
Epoch 8/10
1318/1318 [==============================] - 323s 245ms/step - loss: 2.4585
Epoch 9/10
1318/1318 [==============================] - 322s 245ms/step - loss: 2.4262
Epoch 10/10
1318/1318 [==============================] - 323s 245ms/step - loss: 2.4189


In [ ]:
query = 'سلام چطوری خوبی چه خبرا داری چی کار [MASK]'
inp = tokenizer(query,return_tensors='tf')
print(inp)
mask_loc = np.where(inp.input_ids.numpy()[0] == 103)[0].tolist()
out = model(inp).logits[0].numpy()
predicted_tokens = np.argmax(out[mask_loc],axis=1).tolist()
tokenizer.decode(predicted_tokens)


{'input_ids': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=
array([[    2,  2640, 11994,  2354,  2299,  2066,  1008,  4509,  2291,
         1959,     4,     3]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 12), dtype=int32, numpy=array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}


''

In [ ]:
from transformers import  pipeline
fill_sentence = pipeline('fill-mask', model=model, tokenizer=tokenizer)
fill_sentence( 'من اگر رفتم [MASK] نمیتونم دیگه بیام بیرون از خونه')



[{'score': 0.35739386081695557,
  'token': 3601,
  'token_str': 'بیرون',
  'sequence': 'من اگر رفتم بیرون نمیتونم دیگه بیام بیرون از خونه'},
 {'score': 0.056170396506786346,
  'token': 13028,
  'token_str': 'سرکار',
  'sequence': 'من اگر رفتم سرکار نمیتونم دیگه بیام بیرون از خونه'},
 {'score': 0.03535359352827072,
  'token': 4710,
  'token_str': 'خونه',
  'sequence': 'من اگر رفتم خونه نمیتونم دیگه بیام بیرون از خونه'},
 {'score': 0.02500774711370468,
  'token': 3827,
  'token_str': 'جایی',
  'sequence': 'من اگر رفتم جایی نمیتونم دیگه بیام بیرون از خونه'},
 {'score': 0.02233332395553589,
  'token': 3012,
  'token_str': 'دیگه',
  'sequence': 'من اگر رفتم دیگه نمیتونم دیگه بیام بیرون از خونه'}]

In [ ]:
!pip install hazm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 892.6/892.6 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 19.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 26.2 MB/s eta 0:00:00
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=2abb53567203ffccbdd1174fd7c8275ce17746ce9fdf3e574e9ab9bcc9384da6
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This

In [ ]:
import string
from hazm import *
def normalize_text(text_input):
    normalized_text = Normalizer().normalize(text_input)
    normalized_text = normalized_text.replace("آ", "ا")
    normalized_text = normalized_text.replace("\u200c", " ")
    text_without_punctuations = "".join([" " + ch + " " if ch in string.punctuation else ch for ch in normalized_text])
    words_list = text_without_punctuations.split()
    normalized = " ".join(words_list)
    return normalized

In [ ]:
import gensim

def is_true(resp, input_str, max_threshold=1500):
    tokenized = tokenizer.tokenize(input_str)
    preds = fill_sentence(' '.join(token[2:] if token[:2] == "##" else " " + token for token in tokenized), top_k=max_threshold)
    preds_str = [pred["token_str"] for pred in preds]
    num_of_occurrence = preds_str.index(resp["raw"]) if resp["raw"] in preds_str else -1
    return resp["raw"] in preds_str, num_of_occurrence, preds_str
import timeit
def evaluate(sent):


    sent = normalize_text(sent)
    text_arr = sent.split()
    start_time = timeit.default_timer()
    for ind, word in enumerate(text_arr):
        resp = {"raw": word, "correct": None, "span": [sent.find(word), sent.find(word) + len(word)]}
        input_str = ' '.join(text_arr[:ind] + ["[MASK]"] + text_arr[ind + 1:])

        if resp["raw"] == "ها":
            continue

        is_valid, num_of_occurrence, predictions = is_true(resp, input_str)

        if not is_valid:
            min_dist = float('inf')
            recom_word = predictions[0]

            for potential in predictions:
                distance = gensim.similarities.fastss.editdist(potential, resp["raw"])
                if distance == 1:
                    recom_word = potential
                    break

                if distance < min_dist:
                    min_dist = distance
                    recom_word = potential

            text_arr[ind] = recom_word

    elapsed_time = timeit.default_timer() - start_time
    print(f"Elapsed Time: {elapsed_time} seconds")

    return ' '.join(text_arr)
# Example usage:
sentence = 'در آزمایش هایی که توسط آزمایشسگاه ها انجام می شود واکنش های شیمیشایی ای رخ میدهد'
result = evaluate(sentence)
print(result)


Elapsed Time: 25.698245313999905 seconds
در ازمایش هایی که توسط ازمایشگاه ها انجام می شود واکنش های شیمیایی ای رخ می دهد


In [ ]:
vocab = tokenizer.get_vocab()
vocab['مدل']

3306